# Notebook para explorar datos y definir acciones a ejecutar en transform.py

In [45]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

# Cargar dfs
df_bibliotecas_path = '../data/bibliotecas/2022-octubre/bibliotecas-07-10-2022.csv'
df_cines_path = '../data/cines/2022-octubre/cines-07-10-2022.csv'
df_museos_path = '../data/museos/2022-octubre/museos-07-10-2022.csv'

bibliotecas_df = pd.read_csv(df_bibliotecas_path, encoding='utf8')
cines_df = pd.read_csv(df_cines_path, encoding='utf8')
museos_df = pd.read_csv(df_museos_path, encoding='utf8')

# Ver tamaños y columnas de cada uno
dfs = [bibliotecas_df, cines_df, museos_df]
for df in dfs:
    display(df.columns)
    display(df.shape)

Index(['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Observacion', 'Categoría',
       'Subcategoria', 'Provincia', 'Departamento', 'Localidad', 'Nombre',
       'Domicilio', 'Piso', 'CP', 'Cod_tel', 'Teléfono', 'Mail', 'Web',
       'Información adicional', 'Latitud', 'Longitud', 'TipoLatitudLongitud',
       'Fuente', 'Tipo_gestion', 'año_inicio', 'Año_actualizacion'],
      dtype='object')

(2017, 25)

Index(['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Observaciones',
       'Categoría', 'Provincia', 'Departamento', 'Localidad', 'Nombre',
       'Dirección', 'Piso', 'CP', 'cod_area', 'Teléfono', 'Mail', 'Web',
       'Información adicional', 'Latitud', 'Longitud', 'TipoLatitudLongitud',
       'Fuente', 'tipo_gestion', 'Pantallas', 'Butacas', 'espacio_INCAA',
       'año_actualizacion'],
      dtype='object')

(329, 26)

Index(['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Observaciones',
       'categoria', 'subcategoria', 'provincia', 'localidad', 'nombre',
       'direccion', 'piso', 'CP', 'cod_area', 'telefono', 'Mail', 'Web',
       'Latitud', 'Longitud', 'TipoLatitudLongitud', 'Info_adicional',
       'fuente', 'jurisdiccion', 'año_inauguracion', 'actualizacion'],
      dtype='object')

(1182, 24)

## A) Transformaciones para master dataset
### 1° Unificar nombres de columnas comunes requeridas entre los 3 dfs

In [46]:
# Columnas para master_df: cod_localidad, id_provincia, id_departamento, categoría, 
# provincia, localidad, nombre, domicilio, código postal, número de teléfono, mail, web

# Cargar nombres de columnas de interés de cada df en mismo orden
columns = ['cod_localidad', 'id_provincia', 'id_departamento', 'categoria', 
                'provincia', 'localidad','nombre','domicilio', 'codigo_postal', 'Cod_tel',
                'Teléfono', 'mail', 'web', 'fuente' ]
bibliotecas_columns = ['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Categoría', 
                        'Provincia', 'Localidad','Nombre','Domicilio', 'CP', 'Cod_tel', 
                        'Teléfono', 'Mail', 'Web', 'Fuente']
cines_columns = ['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Categoría', 'Provincia', 
                    'Localidad','Nombre','Dirección', 'CP', 'cod_area', 'Teléfono', 'Mail', 
                    'Web', 'Fuente']
museos_columns = ['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'categoria', 'provincia', 
                    'localidad','nombre','direccion', 'CP', 'cod_area', 'telefono', 
                    'Mail', 'Web', 'fuente']

# Renombrar columnas en cada df para que todos sean iguales y se pueda concatenar
bibliotecas_df.rename(columns=dict(zip(bibliotecas_columns, columns)), inplace=True)
cines_df.rename(columns=dict(zip(cines_columns, columns)), inplace=True)
museos_df.rename(columns=dict(zip(museos_columns, columns)), inplace=True)

# Concatenar dfs en master y ver resultados
master_df = pd.concat([bibliotecas_df[columns], cines_df[columns], museos_df[columns]])
display(master_df.columns, master_df.shape)
master_df.sample(5)

Index(['cod_localidad', 'id_provincia', 'id_departamento', 'categoria',
       'provincia', 'localidad', 'nombre', 'domicilio', 'codigo_postal',
       'Cod_tel', 'Teléfono', 'mail', 'web', 'fuente'],
      dtype='object')

(3528, 14)

,cod_localidad,id_provincia,id_departamento,categoria,provincia,localidad,nombre,domicilio,codigo_postal,Cod_tel,Teléfono,mail,web,fuente
1064,46119010,46,46119,Bibliotecas Populares,La Rioja,Alpasinche,Biblioteca Popular Pastor de la Colina Villafañe,Ruta Nacional Nro. 40,5325.0,3825.0,422560.0,s/d,NaN,CONABIP
12,6105060,6,6105,Bibliotecas Populares,Buenos Aires,Bolívar,Biblioteca Popular Bernardino Rivadavia,Las Heras 80,NaN,2314.0,420626.0,bib_rivadavia@yahoo.com.ar,NaN,CONABIP
663,86035090,86,86035,Bibliotecas Populares,Santiago del Estero,La Banda,Biblioteca Popular Narciso Laprida,Laprida 434,NaN,385.0,4270932.0,s/d,NaN,CONABIP
1638,6490010,6,6490,Bibliotecas Populares,Buenos Aires,Ingeniero Budge,Biblioteca Popular Leopoldo Marechal,Campana 1629,1832.0,11.0,42851819.0,bibliotecamarechal@hotmail.com,NaN,CONABIP
1914,2000010,2,2000,Bibliotecas Populares,Ciudad Autónoma de Buenos Aires,Ciudad de Buenos Aires,Biblioteca Popular Creando Horizontes Educativos,Albarellos 2825,1419.0,11.0,38542459.0,s/d,NaN,CONABIP


### 2° Unificar cod_tel y telefono en una única columna

In [3]:
display(master_df['Cod_tel'].dtype, master_df['Teléfono'].dtype)
master_df[['Cod_tel', 'Teléfono']].sample(9)

dtype('float64')

dtype('O')

,Cod_tel,Teléfono
310,2257.0,422022.0
1228,11.0,47687377.0
1547,3404.0,471037.0
1484,NaN,NaN
2000,NaN,NaN
689,3755.0,425832.0
684,2954.0,381 998
736,11.0,42257383.0
25,2344.0,481246.0


In [4]:
# Cod_tel es de tipo float por lo que los nros tienen agregado .0 al final, y hay valores nulos
# Teléfono es tipo str, tiene valores con espacios en blanco, pero también tiene algunos con .0 al final y valores nulos
# La nueva columna será str para preservar nulos, y se creará uniendo los dos campos (siempre y cuando haya valor en Teléfono)
# separados por un espacio (para seguir formato presentado), previa eliminación de los .0

def make_phone_number(row):

    if not pd.isnull(row['Teléfono']):
        # convierto a str
        tel = str(row['Teléfono']).strip()
        # elimino .0 final si tiene
        if tel.find('.0') != -1:
            tel = tel[:tel.find('.0')]
        # chequeo código para ver si debo concatenar o va solo nro de telefono sin cod area
        if not pd.isnull(row['Cod_tel']):
            cod = str(row['Cod_tel']).strip()
            if cod.find('.0') != -1:
                cod = cod[:cod.find('.0')]
            row['telefono'] = cod + ' ' + tel
        else:
            row['telefono'] = tel
    else:
    # si no tengo número de teléfono no me interesa si hay código de área 
        row['telefono'] = np.nan
    return row    

master_df = master_df.apply(make_phone_number, axis='columns')

master_df[['Cod_tel', 'Teléfono', 'telefono']].sample(12)
master_df.drop(['Cod_tel', 'Teléfono'], inplace=True, axis='columns')
columns.remove('Cod_tel')
columns.remove('Teléfono')

### Explorar datos por columna

In [5]:
# Explorar datos, chequeando nulos, valores únicos y distribución de valores
total_rows = master_df.shape[0]
for column in columns:
    display(f'---------------------------- Columna {column} - {master_df[column].dtype} ----------------------------')
    no_nulos = master_df[column].count()
    unique_values_count = len(master_df[column].unique())
    display(f'Total: {total_rows}  |  Nulos: {total_rows - no_nulos}  |  No nulos: {no_nulos} | Únicos: {unique_values_count}')
    if unique_values_count > 30:
        display(f'Top Distribución de valores:')
        display(master_df[column].value_counts().iloc[:15])
    else:
        display(f'Valores únicos:')
        display(master_df[column].unique())
        display(f'Distribución de valores:')
        display(master_df[column].value_counts())


'---------------------------- Columna cod_localidad - int64 ----------------------------'

'Total: 3528  |  Nulos: 0  |  No nulos: 3528 | Únicos: 1197'

'Top Distribución de valores:'

2000010     210
14014010     92
82063170     72
6441030      63
82084270     61
66028050     50
22140060     46
58035070     37
86049110     31
50007010     31
70028010     30
90084010     27
6056010      24
6427010      22
38021060     21
Name: cod_localidad, dtype: int64

'---------------------------- Columna id_provincia - int64 ----------------------------'

...

s/d                                         561
rmazzutti@hoyts.com.ar                       11
iventura@cinemark.com.ar                      9
info@atlascines.com.ar                        5
fharari@hotmail.com.ar                        4
diversidad.cultural@santafeciudad.gov.ar      4
admin@lastipasboulevard.com.ar                3
nuevocineteatrorex@gmail.com                  3
cluduena@grupodinosaurio.com                  3
rabdenur@gmail.com                            3
ericontadoralinares@gmail.com                 3
naiarg@naiarg.com.ar                          3
gabriel.guralnik@gmail.com                    2
york.cultura@vicentelopez.gov.ar              2
museoesperanza@esperanza.gov.ar               2
Name: mail, dtype: int64

'---------------------------- Columna web - object ----------------------------'

'Total: 3528  |  Nulos: 3124  |  No nulos: 404 | Únicos: 308'

'Top Distribución de valores:'

http://www.cinemacenter.com.ar/            17
s/d                                        16
https://www.cinemarkhoyts.com.ar/          11
https://www.cinemarkhoyts.com.ar           10
https://www.villagecines.com/               8
cineatlasweb.com.ar                         7
https://www.multiplex.com.ar/               6
https://www.todoshowcase.com/               6
http://www.cinesunstar.com/                 6
www.ara.mil.ar                              4
www.enteculturaltucuman.gov.ar              4
http://cinema3dlarioja.com/                 3
http://www.cinemalaplata.com                3
www.culturarionegro.gov.ar                  3
https://www.cinesdinomall.com.ar/Inicio     2
Name: web, dtype: int64

#### Resultados de de la exploración
- IdProvincia tiene 24 valores únicos pero Provincia tiene 27 valores únicos. Hay 3 provincias que tienen dos nombres cada una y se deben unificar: 
1. Neuquén y Neuquén\xa0, 
2. Tierra del Fuego y Tierra del Fuego, Antártida e Islas del Atlántico Sur
3. Santa Fe y Santa Fé
- Domicilio: tiene 18 nulos + 30 'sin dirección', convertir a nulos
- Web tiene 3124 nulos y 16 valores 's/d', convertir a nulos
- Mail tiene 918 nulos y 561 's/d', convertir a nulos
- CP es string pero tiene valores con .0 al final, eliminar .0

### 3° Unificar nombres de Provincia

In [6]:
master_df[['id_provincia', 'provincia', 'categoria']].groupby(['id_provincia', 'provincia']).count().sort_values(by='id_provincia')

,,categoria
id_provincia,provincia,
2,Ciudad Autónoma de Buenos Aires,210
6,Buenos Aires,801
10,Catamarca,74
14,Córdoba,346
18,Corrientes,80
22,Chaco,107
26,Chubut,102
30,Entre Ríos,128
34,Formosa,34


In [7]:
def unify_province_names(row):
    if row['provincia'] == 'Neuquén\xa0':
        row['provincia'] = 'Neuquén'
    elif row['provincia'] == 'Santa Fe':
        row['provincia'] = 'Santa Fé'
    elif row['provincia'] == 'Tierra del Fuego':
        row['provincia'] = 'Tierra del Fuego, Antártida e Islas del Atlántico Sur'
    return row
master_df = master_df.apply(unify_province_names, axis='columns')

master_df[['id_provincia', 'provincia', 'categoria']].groupby(['id_provincia', 'provincia']).count().sort_values(by='id_provincia')

,,categoria
id_provincia,provincia,
2,Ciudad Autónoma de Buenos Aires,210
6,Buenos Aires,801
10,Catamarca,74
14,Córdoba,346
18,Corrientes,80
22,Chaco,107
26,Chubut,102
30,Entre Ríos,128
34,Formosa,34


- Se detecta una row que tiene Salta con codigo 58, que corresponde a Neuquén. Cambiar a 66
### 4° Cambiar cód de Salta en row erronea

In [8]:
# Chequear si lo que está incorrecto es el codigo o el nombre de la pcia.
master_df.loc[(master_df['provincia'] == 'Salta') & (master_df['id_provincia'] == 58)]

,cod_localidad,id_provincia,id_departamento,categoria,provincia,localidad,nombre,domicilio,codigo_postal,mail,web,telefono
860,58035070,58,58035,Espacios de Exhibición Patrimonial,Salta,Salta,Museo Catedralicio,Av. Belgrano 589,NaN,NaN,NaN,NaN


In [9]:
master_df.loc[(master_df['provincia'] == 'Salta') & (master_df['id_provincia'] == 58), 'id_provincia'] = 66

#chequear con group by
master_df[['id_provincia', 'provincia', 'categoria']].groupby(['id_provincia', 'provincia']).count().sort_values(by='id_provincia')

,,categoria
id_provincia,provincia,
2,Ciudad Autónoma de Buenos Aires,210
6,Buenos Aires,801
10,Catamarca,74
14,Córdoba,346
18,Corrientes,80
22,Chaco,107
26,Chubut,102
30,Entre Ríos,128
34,Formosa,34


- Antes de unificar nulos, chequear que no haya 0s o espacios en blanco en ninguna columna

In [10]:
from pandas.api.types import is_string_dtype, is_numeric_dtype

columns.append('telefono')

for column in columns:
    if is_string_dtype(master_df[column]):
        # Para str chequeo espacio en blanco y vacio
        empty_count = master_df.loc[master_df[column] == '', column].count()
        whitespace_count = master_df.loc[master_df[column] == ' ', column].count()        
        display(f'{column}: empty {empty_count}, whitespace {whitespace_count}')
    elif is_numeric_dtype(master_df[column]):
        # Para numeric chequeo 0
        zero_count = master_df.loc[master_df[column] == 0, column].count()
        display(f'{column}: zero {zero_count}')

'cod_localidad: zero 0'

'id_provincia: zero 0'

'id_departamento: zero 0'

'categoria: empty 0, whitespace 0'

'provincia: empty 0, whitespace 0'

...

'domicilio: empty 0, whitespace 0'

'codigo_postal: empty 0, whitespace 0'

'mail: empty 0, whitespace 0'

'web: empty 0, whitespace 0'

'telefono: empty 0, whitespace 0'

### 5° Unificar nulos

In [11]:
#Domicilio: tiene 18 nulos + 30 'Sin dirección', convertir a nulos
#Web tiene 3124 nulos y 16 valores 's/d', convertir a nulos
#Mail tiene 918 nulos y 561 's/d', convertir a nulos

def convert_text_nulls_to_nan(row):
    nulls = ['sin dirección', 's/d']
    if str(row['domicilio']).strip().lower() in nulls:
        row['domicilio'] = np.nan
    if str(row['web']).strip().lower() in nulls:
        row['web'] = np.nan
    if str(row['mail']).strip().lower() in nulls:
        row['mail'] = np.nan
    return row

master_df = master_df.apply(convert_text_nulls_to_nan, axis='columns')
# Check
display(master_df['domicilio'].value_counts())
display(master_df['mail'].value_counts())
display(master_df['web'].value_counts())



Sarmiento               10
San Martín               9
25 de mayo               5
9 de julio               5
Sarmiento 450            4
                        ..
Junín 2238               1
Chubut y O.del Sur       1
Ruta 36 Km 3             1
Av. José Roberti 363     1
Ernesto Padilla S/N      1
Name: domicilio, Length: 3367, dtype: int64

rmazzutti@hoyts.com.ar                      11
iventura@cinemark.com.ar                     9
info@atlascines.com.ar                       5
diversidad.cultural@santafeciudad.gov.ar     4
fharari@hotmail.com.ar                       4
                                            ..
dirrecioncultura@52.coopenet.com.ar          1
biblio2945amenabar@hotmail.com               1
bibliotecahoracioquiroga@live.com            1
bibliotecalugones@bombal.net.ar              1
museoejercitotucuman@hotmail.com             1
Name: mail, Length: 1978, dtype: int64

http://www.cinemacenter.com.ar/                                                       17
https://www.cinemarkhoyts.com.ar/                                                     11
https://www.cinemarkhoyts.com.ar                                                      10
https://www.villagecines.com/                                                          8
cineatlasweb.com.ar                                                                    7
                                                                                      ..
https://twitter.com/cineteatroalfa                                                     1
https://www.facebook.com/pg/CineAHZ/about/?ref=page_internal                           1
https://www.facebook.com/pg/Cines-Avenida-112830095434025/about/?ref=page_internal     1
https://www.facebook.com/cine.italia/                                                  1
www.casaindependencia.com.ar                                                           1
Name: web, Length: 30

### 6° Quitar .0 de CP

In [12]:
display(master_df['codigo_postal'].dtype)
master_df['codigo_postal'].sample(6)

dtype('O')

25         5700
299    C1416AAI
390    C1027AAO
767      3025.0
177      5777.0
367         NaN
Name: codigo_postal, dtype: object

In [13]:
master_df['codigo_postal'] = master_df['codigo_postal'].map(lambda x: str(x)[:-2] if str(x).find('.0') != -1 else x)
master_df['codigo_postal'].sample(6)

1293    9311
1661    4427
270     3044
745     5755
1853    3013
766     5501
Name: codigo_postal, dtype: object

## B) Transformaciones para resumen dataset

### 1° Cantidad de registros totales por categoria

In [14]:
# Cantidad de registros totales por categoría
df_totales_categoria = master_df[['id_provincia', 'categoria']].groupby('categoria').count()
df_resumen = pd.DataFrame(
    {
        'etiqueta': ['categoria']*3,
        'valor': df_totales_categoria.index,
        'total': df_totales_categoria['id_provincia']
    }
)

df_resumen.reset_index(inplace=True)
df_resumen.drop(['categoria'], inplace=True, axis='columns')
df_resumen

,etiqueta,valor,total
0,categoria,Bibliotecas Populares,2017
1,categoria,Espacios de Exhibición Patrimonial,1182
2,categoria,Salas de cine,329


### 2° Cantidad de registros totales por fuente

In [49]:
# Cantidad de registros totales por fuente

df_totales_categoria = master_df[['id_provincia', 'fuente']].groupby('fuente').count()
df_resumen2 = pd.DataFrame(
    {
        'etiqueta': ['fuente']*df_totales_categoria.shape[0],
        'valor': df_totales_categoria.index,
        'total': df_totales_categoria['id_provincia']
    } 
)
df_resumen2.reset_index(inplace=True)
df_resumen2.drop(['fuente'], inplace=True, axis='columns')

df_resumen2

,etiqueta,valor,total
0,fuente,CNMLH - Enlace SInCA,1
1,fuente,CNMLH - Ente Cultural de Tucumán,1
2,fuente,CONABIP,2010
3,fuente,DNPyM,1048
4,fuente,Dirección Provincial de Patrimonio Cultural - Enlace SInCA,1
5,fuente,Dirección de Cultura de Villa Allende,2
6,fuente,Dirección de Cultura y de Educación de la Municipalidad de Unquillo,1
7,fuente,Gob. Pcia.,1
8,fuente,Gobierno de la Provincia,14
9,fuente,Gobierno de la Provincia de Chubut / SInCA 2013,3


### 3° Cantidad de registros por provincia y categoria

In [16]:
# Cantidad de registros por provincia y categoría
df_totales_prov_categoria = master_df[['id_provincia', 'provincia', 'categoria']].groupby(['provincia','categoria']).count()
display(df_totales_prov_categoria)
values = ['_'.join(row) for row in df_totales_prov_categoria.index]

id_provincia
provincia                                             categoria                                       
Buenos Aires                                          Bibliotecas Populares                        543
                                                      Espacios de Exhibición Patrimonial           152
                                                      Salas de cine                                106
Catamarca                                             Bibliotecas Populares                         38
                                                      Espacios de Exhibición Patrimonial            32
...                                                                                                ...
Tierra del Fuego, Antártida e Islas del Atlántico Sur Espacios de Exhibición Patrimonial            14
                                                      Salas de cine                                  3
Tucumán                                               Bibliotecas Populares                         37
                                                      Espacios de Exhibición Patrimonial            26
                                                      Salas de cine                                  8

[72 rows x 1 columns]

In [17]:
df_resumen3 = pd.DataFrame(
        {
        'etiqueta': ['provincia_categoria']*df_totales_prov_categoria.shape[0],
        'valor': ['_'.join(row) for row in df_totales_prov_categoria.index],
        'total': df_totales_prov_categoria['id_provincia']
    }
)
df_resumen3.reset_index(inplace=True)
df_resumen3.drop(['provincia', 'categoria'], inplace=True, axis='columns')
df_resumen3

,etiqueta,valor,total
0,provincia_categoria,Buenos Aires_Bibliotecas Populares,543
1,provincia_categoria,Buenos Aires_Espacios de Exhibición Patrimonial,152
2,provincia_categoria,Buenos Aires_Salas de cine,106
3,provincia_categoria,Catamarca_Bibliotecas Populares,38
4,provincia_categoria,Catamarca_Espacios de Exhibición Patrimonial,32
...,...,...,...
67,provincia_categoria,"Tierra del Fuego, Antártida e Islas del Atlántico Sur_Espacios de Exhibición Patrimonial",14
68,provincia_categoria,"Tierra del Fuego, Antártida e Islas del Atlántico Sur_Salas de cine",3
69,provincia_categoria,Tucumán_Bibliotecas Populares,37
70,provincia_categoria,Tucumán_Espacios de Exhibición Patrimonial,26


In [18]:
# Concat all three to make the final summary df
df_summary = pd.concat([df_resumen, df_resumen2, df_resumen3], axis='rows', copy=True)
df_summary

,etiqueta,valor,total
0,categoria,Bibliotecas Populares,2017
1,categoria,Espacios de Exhibición Patrimonial,1182
2,categoria,Salas de cine,329
0,fuente,bibliotecas,2017
1,fuente,cines,329
...,...,...,...
67,provincia_categoria,"Tierra del Fuego, Antártida e Islas del Atlántico Sur_Espacios de Exhibición Patrimonial",14
68,provincia_categoria,"Tierra del Fuego, Antártida e Islas del Atlántico Sur_Salas de cine",3
69,provincia_categoria,Tucumán_Bibliotecas Populares,37
70,provincia_categoria,Tucumán_Espacios de Exhibición Patrimonial,26


## C) Transforms para dataset de cines

In [21]:
#Provincia, Cantidad de pantallas, Cantidad de butacas, Cantidad de espacios INCAA
cines_df.sample(3)

,cod_localidad,id_provincia,id_departamento,Observaciones,categoria,provincia,Departamento,localidad,nombre,domicilio,...,Información adicional,Latitud,Longitud,TipoLatitudLongitud,Fuente,tipo_gestion,Pantallas,Butacas,espacio_INCAA,año_actualizacion
202,50077040,50,50077,NaN,Salas de cine,Mendoza,Malargue,Malargüe,Sala Maitén,Ruta 40 Norte Y Pje Orteguina,...,NaN,-3.548.035.000,-6.958.467.000,Localización precisa,INCAA / SInCA,Público municipal,1,160,NaN,2018
321,6357110,6,6357,NaN,Salas de cine,Buenos Aires,General Pueyrredon,Mar del Plata,Cinemacenter,Rivadavia 3050,...,NaN,-3.799.796.500,-5.755.201.900,Localización precisa,INCAA / SInCA,Privado comercial,2,487,NaN,2018
226,14021290,14,14021,NaN,Salas de cine,Córdoba,Colon,Villa Allende,Showcase,Rio De Janeiro 1725,...,NaN,-3.129.374.000,-6.429.076.600,Localización precisa,INCAA / SInCA,Privado comercial,3,563,NaN,2018


#### Explorar columnas a resumir

In [42]:
total_rows = cines_df.shape[0]
for column in ['Pantallas', 'Butacas', 'espacio_INCAA']:
    display(f'---------------------------- Columna {column} - {cines_df[column].dtype} ----------------------------')
    no_nulos = cines_df[column].count()
    unique_values_count = len(cines_df[column].unique())
    display(f'Total: {total_rows}  |  Nulos: {total_rows - no_nulos}  |  No nulos: {no_nulos} | Únicos: {unique_values_count}')
    if unique_values_count > 30:
        display(f'Top Distribución de valores:')
        display(cines_df[column].value_counts().iloc[:15])
    else:
        display(f'Valores únicos:')
        display(cines_df[column].unique())
        display(f'Distribución de valores:')
        display(cines_df[column].value_counts())

'---------------------------- Columna Pantallas - int64 ----------------------------'

'Total: 329  |  Nulos: 0  |  No nulos: 329 | Únicos: 15'

'Valores únicos:'

array([ 5,  1,  2,  3,  4,  6, 10,  8, 16,  9,  7, 13, 14, 12, 17],
      dtype=int64)

'Distribución de valores:'

...

'Total: 329  |  Nulos: 266  |  No nulos: 63 | Únicos: 3'

'Valores únicos:'

array([nan, 'si', 'SI'], dtype=object)

'Distribución de valores:'

si    52
SI    11
Name: espacio_INCAA, dtype: int64

#### 1° Groupby provincia, sumar pantallas

In [24]:
# Cantidad de pantallas por provincia
df_pantallas = cines_df[['Pantallas', 'provincia']].groupby('provincia').sum()

df_pantallas

,Pantallas
provincia,
Buenos Aires,358
Catamarca,12
Chaco,14
Chubut,10
Ciudad Autónoma de Buenos Aires,153
Corrientes,17
Córdoba,105
Entre Ríos,17
Formosa,4


#### 2° Groupby provincia, sumar butacas

In [25]:
# Cantidad de butacas por provincia
df_butacas = cines_df[['Butacas', 'provincia']].groupby('provincia').sum()

df_butacas

,Butacas
provincia,
Buenos Aires,93112
Catamarca,3200
Chaco,2469
Chubut,2682
Ciudad Autónoma de Buenos Aires,31386
Corrientes,3370
Córdoba,20799
Entre Ríos,4086
Formosa,1184


#### 3° Groupby provincia, contar espacio_INCAA (corregir 0 primero)

In [41]:
# Cantidad de espacios incaa por provincia
# Primero cambiamos el 0 por Nan para que no lo cuente
#cines_df.loc[cines_df['espacio_INCAA']=='0', 'espacio_INCAA'] = np.nan
cines_df['espacio_INCAA'] = cines_df['espacio_INCAA'].map(lambda x: np.nan if x=='0' else x)
df_incaa = cines_df[['espacio_INCAA', 'provincia']].groupby('provincia').count()

df_incaa

,espacio_INCAA
provincia,
Buenos Aires,20
Catamarca,1
Chaco,1
Chubut,4
Ciudad Autónoma de Buenos Aires,3
Corrientes,1
Córdoba,2
Entre Ríos,2
Formosa,1


In [44]:
# concat dfs by columns
df_resumen_cine = pd.concat([df_pantallas, df_butacas, df_incaa], axis='columns', copy=True)
df_resumen_cine.rename(columns={
        'Pantallas': 'cant_pantallas', 
        'Butacas': 'cant_butacas',
        'espacio_INCAA': 'cant_espacios_incaa'
        }, inplace=True)
df_resumen_cine.reset_index(inplace=True)
df_resumen_cine

,provincia,cant_pantallas,cant_butacas,cant_espacios_incaa
0,Buenos Aires,358,93112,20
1,Catamarca,12,3200,1
2,Chaco,14,2469,1
3,Chubut,10,2682,4
4,Ciudad Autónoma de Buenos Aires,153,31386,3
5,Corrientes,17,3370,1
6,Córdoba,105,20799,2
7,Entre Ríos,17,4086,2
8,Formosa,4,1184,1
9,Jujuy,5,2277,2
